In [ ]:
# default_exp core

# core

> Core routines

In [ ]:
#hide
from nbdev.showdoc import *

In [ ]:
#export 
import torch
import torch.nn as nn
import torch.nn.functional  as F 
import torchaudio
from torchaudio import transforms as T
import tqdm
from pathlib import Path
import yaml
import os

## Audio utils

In [ ]:
#export
def is_silence(
    audio,       # torch tensor of multichannel audio
    thresh=-70,  # threshold in dB below which we declare to be silence
    ):
    "checks if entire clip is 'silence' below some dB threshold"
    dBmax = 20*torch.log10(torch.flatten(audio.abs()).max()).cpu().numpy()
    return dBmax < thresh

In [ ]:
# code tests
x = torch.ones((2,10))
assert not is_silence(1e-3*x) # not silent
assert is_silence(1e-5*x) # silent
assert is_silence(1e-3*x, thresh=-50) # higher thresh

In [ ]:
#|export
def load_audio(
    filename:str,     # file to load
    sr=48000,         # sample rate to read/resample at 
    )->torch.tensor:
    "this loads an audio file as a torch tensor"
    audio, in_sr = torchaudio.load(filename)
    if in_sr != sr:
        print(f"Resampling {filename} from {in_sr} Hz to {sr} Hz",flush=True)
        resample_tf = T.Resample(in_sr, sr)
        audio = resample_tf(audio)
    return audio

In [ ]:
#|export
def makedir(
    path:str,          # directory or nested directory
    ):
    "creates directories where they don't exist"
    if os.path.isdir(path): return  # don't make it if it already exists
    #print(f"  Making directory {path}")
    try:
        os.makedirs(path)  # recursively make all dirs named in path
    except:                # don't really care about errors
        pass